In [1]:
import papersys
from papersys.database.manager import PaperManager
from papersys.database.name import *
from papersys.database.schema import PAPER_METADATA_SCHEMA, pa2pl_schema

In [2]:
manager = PaperManager()
manager.drop_metadata_table()
manager.create_metadata_table()
manager.db["paper_metadata"]


2025-10-23 23:26:21.340 | INFO     | papersys.database.manager:__init__:11 - Connecting to database at data/sample-lancedb
2025-10-23 23:26:21.343 | ERROR    | papersys.database.manager:drop_metadata_table:35 - Error dropping metadata table: Table 'paper_metadata' was not found


LanceTable(name='paper_metadata', version=4, _conn=LanceDBConnection(uri='/home/lyk/code/papersys/data/sample-lancedb'))

In [16]:
import polars as pl
# 空格切分成列表
df = pl.read_ndjson(
    "data/arxiv-metadata-oai-snapshot.jsonl",
).with_columns(pl.col("categories").str.split(" ").alias("categories"))
df.head()

id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed
str,str,str,str,str,str,str,str,list[str],str,str,list[struct[2]],str,list[list[str]]
"""0704.0001""","""Pavel Nadolsky""","""C. Bal\'azs, E. L. Berger, P. …","""Calculation of prompt diphoton…","""37 pages, 15 figures; publishe…","""Phys.Rev.D76:013009,2007""","""10.1103/PhysRevD.76.013009""","""ANL-HEP-PR-07-12""","[""hep-ph""]",null,""" A fully differential calcula…","[{""v1"",""Mon, 2 Apr 2007 19:18:42 GMT""}, {""v2"",""Tue, 24 Jul 2007 20:10:27 GMT""}]","""2008-11-26""","[[""Balázs"", ""C."", """"], [""Berger"", ""E. L."", """"], … [""Yuan"", ""C. -P."", """"]]"
"""0704.0002""","""Louis Theran""","""Ileana Streinu and Louis Thera…","""Sparsity-certifying Graph Deco…","""To appear in Graphs and Combin…",null,null,null,"[""math.CO"", ""cs.CG""]","""http://arxiv.org/licenses/none…",""" We describe a new algorithm,…","[{""v1"",""Sat, 31 Mar 2007 02:26:18 GMT""}, {""v2"",""Sat, 13 Dec 2008 17:26:00 GMT""}]","""2008-12-13""","[[""Streinu"", ""Ileana"", """"], [""Theran"", ""Louis"", """"]]"
"""0704.0003""","""Hongjun Pan""","""Hongjun Pan""","""The evolution of the Earth-Moo…","""23 pages, 3 figures""",null,null,null,"[""physics.gen-ph""]",null,""" The evolution of Earth-Moon …","[{""v1"",""Sun, 1 Apr 2007 20:46:54 GMT""}, {""v2"",""Sat, 8 Dec 2007 23:47:24 GMT""}, {""v3"",""Sun, 13 Jan 2008 00:36:28 GMT""}]","""2008-01-13""","[[""Pan"", ""Hongjun"", """"]]"
"""0704.0004""","""David Callan""","""David Callan""","""A determinant of Stirling cycl…","""11 pages""",null,null,null,"[""math.CO""]",null,""" We show that a determinant o…","[{""v1"",""Sat, 31 Mar 2007 03:16:14 GMT""}]","""2007-05-23""","[[""Callan"", ""David"", """"]]"
"""0704.0005""","""Alberto Torchinsky""","""Wael Abu-Shammala and Alberto …","""From dyadic $\Lambda_{\alpha}$…",null,"""Illinois J. Math. 52 (2008) no…",null,null,"[""math.CA"", ""math.FA""]",null,""" In this paper we show how to…","[{""v1"",""Mon, 2 Apr 2007 18:09:58 GMT""}]","""2013-10-15""","[[""Abu-Shammala"", ""Wael"", """"], [""Torchinsky"", ""Alberto"", """"]]"


In [4]:
manager.db["paper_metadata"].add(df)

AddResult(version=5)

In [5]:
manager.db["paper_metadata"].head()

pyarrow.Table
id: string
submitter: string
authors: string
title: string
comments: string
journal_ref: string
doi: string
report_no: string
categories: string
license: string
abstract: string
update_date: date32[day]
----
id: [["0704.0001","0704.0002","0704.0003","0704.0004","0704.0005"]]
submitter: [["Pavel Nadolsky","Louis Theran","Hongjun Pan","David Callan","Alberto Torchinsky"]]
authors: [["C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-P. Yuan","Ileana Streinu and Louis Theran","Hongjun Pan","David Callan","Wael Abu-Shammala and Alberto Torchinsky"]]
title: [["Calculation of prompt diphoton production cross sections at Tevatron and
  LHC energies","Sparsity-certifying Graph Decompositions","The evolution of the Earth-Moon system based on the dark matter field
  fluid model","A determinant of Stirling cycle numbers counts unlabeled acyclic
  single-source automata","From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\alpha}$"]]
comments: [["37 pages, 15 figures; published version","To app

In [14]:
table = manager.db["paper_metadata"]

results = table.search().where(
    # id 以 19 开头，category 包含 cs.LG，随机排序后取前 5 条
    "(id LIKE '19%') AND (categories like '%cs.LG%')"
).to_polars().sample(n=5, seed=42)

results


id,submitter,authors,title,comments,journal_ref,doi,report_no,categories,license,abstract,update_date
str,str,str,str,str,str,str,str,str,str,str,date
"""1911.00957""","""Iacopo Masi""","""Iacopo Masi, Joe Mathai, Wael …","""Towards Learning Structure via…","""To appear in the IEEE/CVF Conf…",null,null,null,"""cs.CV cs.LG eess.IV""","""http://arxiv.org/licenses/none…",""" Face segmentation is the tas…",2020-03-31
"""1905.10899""","""Ernest Ryu""","""Ernest K. Ryu and Kun Yuan and…","""ODE Analysis of Stochastic Gra…",null,null,null,null,"""cs.LG stat.ML""","""http://arxiv.org/licenses/none…",""" Despite remarkable empirical…",2020-10-13
"""1912.11027""","""William Lotter""","""William Lotter, Abdul Rahman D…","""Robust breast cancer detection…",null,null,null,null,"""eess.IV cs.CV cs.LG""","""http://arxiv.org/licenses/none…",""" Breast cancer remains a glob…",2019-12-30
"""1910.01545""","""William Guss""","""William H. Guss, Ruslan Salakh…","""On Universal Approximation by …","""12 pages""",null,null,null,"""cs.LG math.FA stat.ML""","""http://creativecommons.org/lic…",""" The study of universal appro…",2019-10-04
"""1910.12958""","""Thibault Sejourne""","""Thibault S\'ejourn\'e, Jean Fe…","""Sinkhorn Divergences for Unbal…",null,null,null,null,"""math.OC cs.LG stat.ML""","""http://arxiv.org/licenses/none…",""" Optimal transport induces th…",2023-01-18


57.3 ms ± 1.44 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
